# Preprocessing des Données Kaggle

Dans le jeu de données Kaggle, l'objectif initial du test est de prédire une semaine, en s'appuyant sur des données couvrant la période de 2013 à 2017. 

Cependant, pour augmenter la portée de notre modèle, nous allons élargir l'ensemble de test afin de tenter de prédire **deux ans**.


In [32]:
import os
import pandas as pd

### 📥 Importation et Prétraitement des Données

In [33]:
# --Chemins--
raw_data_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'data', 'raw')
processed_data_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'data', 'processed')
testing_path = os.path.join(raw_data_path, 'test.csv')
training_path = os.path.join(raw_data_path, 'train.csv')
# --Tables--
train_df = pd.read_csv(training_path)
train_df['date'] = pd.to_datetime(train_df['date'], format='%Y-%m-%d')

test_df = pd.read_csv(testing_path)
test_df['date'] = pd.to_datetime(test_df['date'], format='%Y-%m-%d')

# --Fusion--
full_df = pd.concat([train_df, test_df], ignore_index=True)
full_df = full_df.drop(columns=['onpromotion'])


In [34]:
full_df.head()

,id,date,store_nbr,family,sales
0,0,2013-01-01,1,AUTOMOTIVE,0.0
1,1,2013-01-01,1,BABY CARE,0.0
2,2,2013-01-01,1,BEAUTY,0.0
3,3,2013-01-01,1,BEVERAGES,0.0
4,4,2013-01-01,1,BOOKS,0.0


### 🔍Faisons quelques vérifications sur les données

In [35]:

# --Vérification des doublons de dates--
duplicate_dates = full_df['date'].duplicated().sum()
print(f"Nombre de dates qui se répètent : {duplicate_dates}")

# --Vérification des valeurs nulles--
missing_values = full_df.isnull().sum()
print(f"Valeurs manquantes :\n{missing_values}")

# --Vérifier si chaque jour apparait--
date_range = pd.date_range(start=full_df['date'].min(), end=full_df['date'].max())
missing_days = set(date_range) - set(full_df['date'])
print(f"Jours manquants : {len(missing_days)}")
if missing_days:
    print(f"Liste des jours manquants : {sorted(missing_days)}")
else:
    print("Tous les jours sont présents dans la plage de dates.")

Nombre de dates qui se répètent : 3027700
Valeurs manquantes :
id               0
date             0
store_nbr        0
family           0
sales        28512
dtype: int64
Jours manquants : 4
Liste des jours manquants : [Timestamp('2013-12-25 00:00:00', freq='D'), Timestamp('2014-12-25 00:00:00', freq='D'), Timestamp('2015-12-25 00:00:00', freq='D'), Timestamp('2016-12-25 00:00:00', freq='D')]


> Chaque année il nous manque Noel, et nous avons des valeurs `null` dans la colonne `sales`

### ✂️ Séparation des données pour le test : **deux ans** de data

In [31]:
# --Diviser les données en ensembles d'entraînement et de test--
training_data = full_df[full_df['date'] <= '2015-12-31']
testing_data = full_df[full_df['date'] >= '2016-01-01']

training_file_path = os.path.join(processed_data_path, 'training.csv')
testing_file_path = os.path.join(processed_data_path, 'testing.csv')

training_data.to_csv(training_file_path, index=False)
testing_data.to_csv(testing_file_path, index=False)